In [51]:
# eec266_project.ipynb
# Author: Mason del Rosario
# Replicate numerical results from 2010 paper, "Maximum Mutual Information Design for MIMO Systems With Imperfect Channel Knowledge" (Ding, Blostein)
import numpy as np
from scipy.linalg import sqrtm

In [63]:
# parameters for system model
n_T = 4 # transmitter antennas
n_R = 4 # receiver antennas
rho_T = 0.9 # correlation constant for R_T
rho_R = 0.5 # correlation constant for R_R
var_E = 0.01 # error variance

In [71]:
# core function: circularly symmetric complex normal distribution
def circ_complex_normal(m, n, mean=None, cov=None):
    if type(mean)==type(None):
        mean = np.zeros(2)
    if type(cov)==type(None):
        cov = np.eye(2)
    # generate i.i.d complex samples
    samp = np.random.multivariate_normal(mean, cov, (m,n))
    # turn into complex numbers
    out = np.zeros((m,n),dtype=complex)
    for i in range(m):
        for j in range(n):
            temp = samp[i,j]
            out[i,j] = temp[0] + 1j*temp[1]
    return out

# core function: exponential correlation model from Aalto, 1995
def exponential_corr(n, rho):
    # inputs:
    # -> n = # of antennas
    # -> rho = correlation constant for exponential correlation model
    R = np.zeros((n,n))
    for i in range(n):
        for j in range(i,n):
            R[i,j] = rho**(np.abs(i-j))
    R = symmetrize(R)
    return R

# helper function: symmetrize
def symmetrize(a):
    return a + a.T - np.diag(a.diagonal())

In [72]:
# exponential correlation matrices
R_T = exponential_corr(n_T,rho_T)
R_R = exponential_corr(n_R,rho_R)
print('R_T:\n{}'.format(R_T))
print('R_R:\n{}'.format(R_R))

# i.i.d. circularly symmetric complex normal matrices for system model
H_w = circ_complex_normal(n_R,n_T)
Hhat_cov = (1-var_E)*np.eye(2)
Hhat_w = circ_complex_normal(n_R,n_T,cov=Hhat_cov)
E_cov = (var_E)*np.eye(2)
E_w = circ_complex_normal(n_R,n_T,cov=Hhat_cov)
print('H_w:\n{}'.format(H_w))
print('Hhat_w:\n{}'.format(Hhat_w))
print('E_w:\n{}'.format(E_w))

# H from correlation matrices and H_w
H = sqrtm(R_R)*H_w*sqrtm(R_T)
print('H:\n{}'.format(H))

# generate white Gaussian noise vector
var_n = 0.01
cov_n = var_n*np.eye(2)
n = circ_complex_normal(1,n_R,cov=cov_n)# noise vector
print('n:\n{}'.format(n))

R_T:
[[1.    0.9   0.81  0.729]
 [0.9   1.    0.9   0.81 ]
 [0.81  0.9   1.    0.9  ]
 [0.729 0.81  0.9   1.   ]]
R_R:
[[1.    0.5   0.25  0.125]
 [0.5   1.    0.5   0.25 ]
 [0.25  0.5   1.    0.5  ]
 [0.125 0.25  0.5   1.   ]]
H_w:
[[ 0.95508583-0.32474036j -1.77871319-0.10730866j  0.24363117-1.84483825j
  -0.92378808-0.75244613j]
 [ 0.31723265+1.47284802j  0.71548664+1.03010179j  2.6622567 -0.57908518j
   0.00353787+0.24685854j]
 [-1.81651792+1.08725338j -0.18807329-0.15888265j -0.08323192+1.64882563j
   0.57379657+0.06597032j]
 [-0.70709838-0.82682916j  0.67525053-0.00580344j  0.78129974-0.29367216j
  -0.26404263+1.04957999j]]
Hhat_w:
[[-1.62726565+1.94970562j -0.71168023+0.39776993j  0.46956629-0.97954763j
  -1.17341856+0.39903826j]
 [-0.24083512+0.38203326j -0.80482237+1.79033595j  0.25530708+1.22972538j
   0.73930805+0.10692947j]
 [-0.25621567-0.90316045j  0.8064989 -1.65813021j  2.00934204+0.02973744j
  -0.40477256+2.01244658j]
 [-1.70628805-0.46183766j -0.62839325+0.5370109j   